In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
import cupy as cp
from numba import cuda, int64
from timeit import default_timer as timer
import simple

cuda.select_device(0)

In [ ]:
def get_df(times: list[int], samples: int):
    if len(times) < samples - 2:
        return "----"
    return str(round((np.sum(times) - min(times) - max(times)) / (samples - 2) * 1000, 2)).replace(".",",")

def comp_tracks(offset: np.array, track: np.array) -> int:
    return np.sum(np.abs(np.subtract(offset, track)))

def calc_cpu(offset: np.array, track: np.array) -> int:
    return np.argmin(np.array([
        comp_tracks(offset, track[i:(i + 1024)]) for i in range(len(track) - 1024)]))

@cuda.jit(device=True)
def diff(a, b):
    return abs(a - b)

@cuda.jit
def calc_gpu(offset, track, res):
    off = cuda.blockIdx.x + cuda.blockIdx.y * cuda.gridDim.y
    t_idx = cuda.threadIdx.x + cuda.threadIdx.y * cuda.blockDim.y

    if off >= res.shape[0] or t_idx >= track.shape[0]:
        return

    res[off, t_idx] = diff(offset[t_idx], track[off + t_idx])

@cuda.jit
def calc_gpu_shared(offset, track, res):
    off = cuda.blockIdx.x + cuda.blockIdx.y * cuda.gridDim.y
    t_idx = cuda.threadIdx.x + cuda.threadIdx.y * cuda.blockDim.y

    if off >= res.shape[0] or t_idx >= track.shape[0]:
        return

    sharedMem = cuda.shared.array(shape=(1), dtype=int64)
    if t_idx == 0:
        sharedMem[0] = 0
    
    cuda.syncthreads()
    cuda.atomic.add(sharedMem, 0, diff(offset[t_idx], track[off + t_idx]))
    cuda.syncthreads()

    if t_idx == 0:
        res[off] = sharedMem[0]

In [ ]:
original = simple.load_sound_files()[0]
original.skip_samples(10000)
test_data = { int(size): { "offset": original.data[1024:2048], "track": original.data[:int(size)], "noise": original.noise[:int(size)] } for size in [5e3, 1e4, 2e4, 5e4, 1e5, 2e5, 5e5]}

data = test_data[10000]
tmp = [comp_tracks(data["offset"], data["track"][i:(i + 1024)]) for i in range(len(data["track"]) - 1024)]
tmp_n = [comp_tracks(data["offset"], data["noise"][i:(i + 1024)]) for i in range(len(data["noise"]) - 1024)]

plt.plot(range(len(data["track"]) - 1024), tmp) 

## Vanilla

In [ ]:
samples = 12
results = {}

for size, data in test_data.items():
    times = []
    for i in range(samples):
        start = timer()
        res = calc_cpu(data["offset"], data["track"])
        end = timer()
        if res != 1024:
            print(f"Bad value for size: {size}")
            break;
        times.append(end - start)
    print(f"{size:>8} = {get_df(times, samples):>8}ms")

## Vanilla with noise

In [ ]:
samples = 12
results = {}

for size, data in test_data.items():
    times = []
    for i in range(samples):
        start = timer()
        res = calc_cpu(data["offset"], data["noise"])
        end = timer()
        if res != 1024:
            print(res)
            print(f"Bad value for size: {size}")
            break;
        times.append(end - start)
    print(f"{size:>8} = {get_df(times, samples):>8}ms")

## CUDA

In [ ]:
threads_per_block = (32, 32)
block_size = int(np.ceil(np.sqrt(original.len - 1024)))
blocks_per_grid = (block_size, block_size)

samples = 13

for size, data in test_data.items():
    times = []
    for i in range(samples):
        start = timer()
        offset_gpu = cuda.to_device(data["offset"])
        track_gpu = cuda.to_device(data["track"])
        res_gpu = cuda.to_device(np.zeros((len(data["track"]) - 1024, 1024), dtype=np.int16))
        calc_gpu[blocks_per_grid, threads_per_block](offset_gpu, track_gpu, res_gpu)
        arg = np.argmin(np.sum(res_gpu.copy_to_host(), axis=1))
        end = timer()
        if i == 0:
            continue

        if res_gpu != 1024:
            print(f"Bad value for size: {size}")
            break;
        times.append(end - start)
    print(f"{size:>8} = {get_df(times, samples - 1):>8}ms")


## CUDA with noise

In [ ]:
threads_per_block = (32, 32)
block_size = int(np.ceil(np.sqrt(original.len - 1024)))
blocks_per_grid = (block_size, block_size)

samples = 13

for size, data in test_data.items():
    times = []
    for i in range(samples):
        start = timer()
        offset_gpu = cuda.to_device(data["offset"])
        track_gpu = cuda.to_device(data["noise"])
        res_gpu = cuda.to_device(np.zeros((len(data["noise"]) - 1024, 1024), dtype=np.int16))
        calc_gpu[blocks_per_grid, threads_per_block](offset_gpu, track_gpu, res_gpu)
        arg = np.argmin(np.sum(res_gpu.copy_to_host(), axis=1))
        end = timer()
        if i == 0:
            continue

        if res_gpu != 1024:
            print(f"Bad value for size: {size}")
            break;
        times.append(end - start)
    print(f"{size:>8} = {get_df(times, samples - 1):>8}ms")

## CUDA with shared memory

In [ ]:
threads_per_block = (32, 32)
block_size = int(np.ceil(np.sqrt(original.len - 1024)))
blocks_per_grid = (block_size, block_size)

samples = 4

for size, data in test_data.items():
    times = []
    for i in range(samples):
        start = timer()
        offset_gpu = cuda.to_device(data["offset"])
        track_gpu = cuda.to_device(data["track"])
        res_gpu = cuda.to_device(np.zeros((len(data["track"]) - 1024), dtype=np.int16))
        calc_gpu_shared[blocks_per_grid, threads_per_block](offset_gpu, track_gpu, res_gpu)
        arg = np.argmin(res_gpu.copy_to_host())
        end = timer()
        if i == 0:
            continue

        if res != 1024:
            print(f"Bad value for size: {size}")
            break;
        times.append(end - start)
    print(f"{size:>8} = {get_df(times, samples - 1):>8}ms")

## FFT comperasion

In [ ]:
sample_size = 50

print("CPU")
for size, data in test_data.items():
    times = []
    for i in range(sample_size):
        start = timer()
        np.fft.fft(data["track"])
        end = timer()
        times.append(end - start)
    print(f"{size:>8} = {get_df(times, sample_size):>8}ms")

print("GPU")
for size, data in test_data.items():
    times = []
    for i in range(sample_size):
        start = timer()
        cp.fft.fft(cp.array(data["track"])).get()
        end = timer()
        times.append(end - start)
    print(f"{size:>8} = {get_df(times, sample_size):>8}ms")